# Text Mining Student Papers: A Computational Exploration

## 1. Install Packages

In [ ]:
#Install os and glob
import glob 
import os

#Install pandas
import pandas as pd

#Install numpy
import numpy as np

#Imports the Natural Language Toolkit, which is necessary to install NLTK packages and libraries
#!pip install nltk
import nltk

#Installs libraries and packages to tokenize text
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
from  nltk.text import ConcordanceIndex

#Installs libraries and packages to clean text
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

#Import matplotlib for visualizations
import matplotlib.pyplot as plt


#Imports spaCy itself, necessary to use features 
#!pip install spaCy
import spacy
#Load the natural language processing pipeline
nlp = spacy.load("en_core_web_sm")
#Load spaCy visualizer
from spacy import displacy

from scipy import stats

import re  # For preprocessing
from time import time  # To time our operations
from collections import defaultdict  # For word frequency
import logging  # Setting up the loggings to monitor gensim

## 2. Import Student Essays and Metadata

### Import Student Essays and Add to DataFrame

In [ ]:
##Get current working directory 
path = os.getcwd()
print(path)

#Change working directory
path = os.chdir("/Users/megankane/Desktop/Texts")

In [ ]:
#Append all txt files to pandas dataframe
filenames = []
data = []
files = [f for f in os.listdir(path) if os.path.isfile(f)]
for f in files:
    if f.endswith('.txt'):
        with open (f, "rb") as myfile:
            filenames.append(myfile.name)
            data.append(myfile.read())
d = {'ID':filenames,'Text':data}
        
essays = pd.DataFrame(d)
essays

In [ ]:
#Remove encoding characters from Text column (b'\xef\xbb\xbf)
essays['Text'] = essays['Text'].apply(lambda x: x.decode('utf-8', errors='ignore'))
essays['Text'] = essays['Text'].astype(str)

#Remove newline characters and put in new column (will need to split paragraphs later)
essays['Text_Newlines'] = essays['Text']
essays['Text'] = essays['Text'].str.replace(r'\s+|\\r', ' ', regex=True) 
essays['Text'] = essays['Text'].str.replace(r'\s+|\\n', ' ', regex=True) 
essays.head()

In [ ]:
#Remove identifying information from ID
#Remove any occurences of "LATE_" from dataset (otherwise will skew ID cleaning)
essays['ID'] = essays['ID'].str.replace(r'LATE_', '', regex=True) 

#Split book on first underscore (_) in ID, keep only text in between first and second underscore (ID number)
start = essays["ID"].str.split("_", expand = True)
essays['ID'] = start[1]
essays['ID'] = essays['ID'].astype(int)
essays

### Import grades and additional metadata to second dataframe

In [ ]:
path = '/Users/megankane/Desktop/CSVs'# use your path

#Upload csvs with essay metadata
all_files = glob.glob(os.path.join(path, "*.csv"))
metadata = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

#Drop header rows(Points Possible) and test student rows (Student, Test)
metadata = metadata[metadata['Student'].str.contains('Points Possible|Student, Test')==False]
metadata

In [ ]:
#Keep only relevant metadata (ID, Section, Final Portfolio Scores)
clean_metadata = metadata[['ID'] + ['Section'] + list(metadata.loc[:, metadata.columns.str.startswith('Final Portfolio (')])]


#Change columns to float as needed (check with df.dtypes())
clean_metadata["Final Portfolio (1Score)"] = pd.to_numeric(clean_metadata["Final Portfolio (1Score)"], downcast="float")
clean_metadata["Final Portfolio (Score)"] = pd.to_numeric(clean_metadata["Final Portfolio (Score)"], downcast="float")
#Want other metadata? Check the columns
#Get all column names 
#for col in metadata.columns:
   # print(col)

In [ ]:
#Replace all NaN values with 0 
clean_metadata = clean_metadata.replace(np.nan, 0)
clean_metadata.head()

In [ ]:
#Create new final portfolio column with all values
#Add values of each column together; values except correct grade will be zero
score_counts = clean_metadata.columns[2:]
clean_metadata['Portfolio_Score'] = clean_metadata[score_counts].sum(axis=1)
clean_metadata['Portfolio_Score']

In [ ]:
#Drop grade columns for individual classes
clean_metadata = clean_metadata[['ID', 'Section', "Portfolio_Score"]]

#Round scores to nearest integer
clean_metadata.Portfolio_Score = clean_metadata.Portfolio_Score.round()
clean_metadata

In [ ]:
#Drop decimal from ID (inconsistent with ID in essay dataframe)
clean_metadata['ID'] = clean_metadata['ID'].astype(float)
clean_metadata['ID'] = clean_metadata['ID'].astype(int)


#Check cleaned DF one more time
clean_metadata.head()

### Merge essays and grade metadata into one dataframe

In [ ]:
#Merge metadata and cleaned essays into new dataframe
#Will only keep rows where both essay and metadata are present
essays_grades_master = clean_metadata.merge(essays,on='ID')

#Print dataframe
essays_grades_master

In [ ]:
#Sort dataframe by grades
essays_grades_master.sort_values(by=['Portfolio_Score'], inplace = True)
essays_grades_master.head()

## 3. Clean Data

### Basic Cleaning with NLTK
Lowercasing, Punctuation Removal, and Stopword Removal

In [ ]:
#Rename dataframe
clean_essay_grades_df = essays_grades_master
clean_essay_grades_df.rename(columns = {"Text_NoHeaders": "Text"}, inplace = True)

#Lowercase all words
clean_essay_grades_df['Lower_Text'] = clean_essay_grades_df['Text'].str.lower()

#Remove punctuation and replace with no space (except periods and hyphens)
clean_essay_grades_df['NoPunct_Text'] = clean_essay_grades_df['Lower_Text'].str.replace(r'[^\w\-\.\'\s]+', '', regex = True)

#Remove periods and replace with space (to prevent incorrect compounds)
clean_essay_grades_df['NoPunct_Text'] = clean_essay_grades_df['NoPunct_Text'].str.replace(r'[^\w\-\'\s]+', ' ', regex = True)

#Remove stopwords
stop_words = set(stopwords.words("english"))
clean_essay_grades_df['NoStops_Text'] = clean_essay_grades_df['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

#Check output
clean_essay_grades_df.head()

## 4. Text Enrichment

Lemmatization, Part-of-Speech Tagging, and Named Entity Recognition with SpaCy

In [ ]:
#Get lemmas
lemma_list = []

# Disable Dependency Parser, and NER since all we want is lemmatizer 
with nlp.disable_pipes('parser', 'ner'):
  #Iterate through each doc object and getlemma, append lemma to list
  for doc in nlp.pipe(clean_essay_grades_df.NoPunct_Text.astype('unicode').values, batch_size=100):
    word_list = []
    for token in doc:
        word_list.append(token.lemma_)
        
    lemma_list.append(word_list)

#Put lemmas in a new column in dataframe
clean_essay_grades_df['Lemma_Text'] = lemma_list
clean_essay_grades_df['Lemma_Text'] = [' '.join(map(str, l)) for l in clean_essay_grades_df['Lemma_Text']]

#Check lemmas
clean_essay_grades_df.head()


In [ ]:
#Get part of speech tags
pos_list = []

# Disable Dependency Parser, and NER since all we want is POS 
with nlp.disable_pipes('parser', 'ner'):
  #Iterate through each doc object and tag POS, append POS to list
  for doc in nlp.pipe(clean_essay_grades_df.NoPunct_Text.astype('unicode').values, batch_size=100):
    word_list = []
    for token in doc:
        word_list.append(token.pos_)
        
    pos_list.append(word_list)

#Put POS in a new column in dataframe
clean_essay_grades_df['POS_Text'] = pos_list
clean_essay_grades_df['POS_Text'] = [' '.join(map(str, l)) for l in clean_essay_grades_df['POS_Text']]

#Check pos tags
clean_essay_grades_df.head()


In [ ]:
#Get named entities
ent_list = []

with nlp.disable_pipes('tagger', 'parser'):
    for doc in nlp.pipe(clean_essay_grades_df.NoPunct_Text.astype('unicode').values, batch_size=100):
        ent_list.append(doc.ents)

#Put NEs in a new column in dataframe
clean_essay_grades_df['NER_Text'] = ent_list
clean_essay_grades_df['NER_Text'] = [' '.join(map(str, l)) for l in clean_essay_grades_df['NER_Text']]

#Check named entities
clean_essay_grades_df.head()


## 5. Paragraph Segmentation

In [ ]:
#We only need one newlines version here
paragraphs_df = clean_essay_grades_df[['Portfolio_Score','ID', 'Text_Newlines']].copy()

#Add ID and score in one column
paragraphs_df['Score_ID'] = 'Score: ' + paragraphs_df['Portfolio_Score'].astype(str) + ', ID: ' + paragraphs_df['ID'].astype(str)

#Check new df
paragraphs_df.head()


In [ ]:
#Count number of paragraphs in each text
paragraph_counts = paragraphs_df['Text_Newlines'].str.count(r'\n')
paragraph_counts

#Append paragraphs counts to dataframe
paragraphs_df["Paragraph_Counts"] = paragraph_counts
paragraphs_df

In [ ]:
#Make new cell each time new paragraph starts 
new = paragraphs_df["Text_Newlines"].str.split(r'\n', expand = True).set_index(paragraphs_df['Score_ID'])

#Flatten dataframe so each chapter is on own row, designated by book and chapter 
paragraphs_df = new.stack().reset_index()
paragraphs_df.columns = ["Score_ID", "Paragraph", "Text"]

#Split score and ID back to own columns
paragraphs_df[['Score','ID']] = paragraphs_df.Score_ID.str.split(", ",expand=True)
paragraphs_df['Score'] = paragraphs_df['Score'].map(lambda x: x.lstrip('Score: '))
paragraphs_df['ID'] = paragraphs_df['ID'].map(lambda x: x.lstrip('ID: '))
paragraphs_df['ID_Paragraph'] = paragraphs_df['ID'].astype(str) + '_' + paragraphs_df['Paragraph'].astype(str)
paragraphs_df

In [ ]:
##Clean paragraphs
##Filter out paragraphs with 5 or less words (headers)
paragraphs_df = paragraphs_df[~paragraphs_df['Text'].str.split().str.len().lt(10)]

## Filter out paragraphs containing "http://", "doi:" , "https://" and "://www" (Works Cited citations)
paragraphs_df = paragraphs_df[~paragraphs_df['Text'].str.contains("http://")]

paragraphs_df = paragraphs_df[~paragraphs_df['Text'].str.contains("https://")]

paragraphs_df = paragraphs_df[~paragraphs_df['Text'].str.contains("://www")]

paragraphs_df = paragraphs_df[~paragraphs_df['Text'].str.contains("www.")]

paragraphs_df = paragraphs_df[~paragraphs_df['Text'].str.contains(".com/")]

paragraphs_df = paragraphs_df[~paragraphs_df['Text'].str.contains("Vol.")]

paragraphs_df = paragraphs_df[~paragraphs_df['Text'].str.contains("doi:")]

paragraphs_df

## 6. Identify Keywords in Context

### Outcome 1: Extracting Rhetorical Analysis Terms and Context

In [ ]:
##Set up new dataframe for keyword frequency counts
rhetorical_keywords_paragraphs_df = paragraphs_df.copy()

#Count number of occurences of rhetorical terms in each paper
pathos_counts = rhetorical_keywords_paragraphs_df['Text'].str.count('pathos')
ethos_counts = rhetorical_keywords_paragraphs_df['Text'].str.count('ethos')
logos_counts = rhetorical_keywords_paragraphs_df['Text'].str.count('logos')

#Append each count to the dataframe
rhetorical_keywords_paragraphs_df['Pathos_Counts'] = pathos_counts
rhetorical_keywords_paragraphs_df["Ethos_Counts"] = ethos_counts
rhetorical_keywords_paragraphs_df["Logos_Counts"] = logos_counts

#Get summ of all term usages
rhetorical_terms = ['Pathos_Counts', 'Ethos_Counts', 'Logos_Counts']
rhetorical_keywords_paragraphs_df['Sum_Terms'] = rhetorical_keywords_paragraphs_df[rhetorical_terms].sum(axis=1)

#Split score and ID back to own columns
rhetorical_keywords_paragraphs_df[['Score','ID']] = rhetorical_keywords_paragraphs_df.Score_ID.str.split(", ",expand=True)
rhetorical_keywords_paragraphs_df['Score'] = rhetorical_keywords_paragraphs_df['Score'].map(lambda x: x.lstrip('Score: '))
rhetorical_keywords_paragraphs_df['ID'] = rhetorical_keywords_paragraphs_df['Score'].map(lambda x: x.lstrip('ID: '))

rhetorical_keywords_paragraphs_df

In [ ]:
#Remove all rows with no rhetorical terms
rhetorical_keywords_paragraphs_df_no_blanks = rhetorical_keywords_paragraphs_df[rhetorical_keywords_paragraphs_df.Sum_Terms > 0]
rhetorical_keywords_paragraphs_df_no_blanks

### Outcome 2: Extracting Citation Practices and Context

In [ ]:
#Get any text inside parentheticals and count of parentheticals and append to dataframe
#https://stackoverflow.com/questions/24696715/regex-for-match-parentheses-in-python
parentheticals = r'(?<=\().*?(?=\))'

#Add new list for parenthetical citations
parenthetical_matches = []
parenthetical_counts = []

#Find all occurences of parenthetical citations in each paragraph of each text
citation_df = paragraphs_df.copy()
for text in citation_df['Text']:
  matches = re.findall(parentheticals, text)
  parenthetical_matches.append(matches)
  parenthetical_counts.append(len(matches))

#Make new column counting all appearances of parentheticals
citation_df["Parentheticals"] = parenthetical_matches
citation_df['Parenthetical_Counts'] = parenthetical_counts

citation_df


In [ ]:
#Remove all rows with no parenthetical terms
citation_df_no_blanks = citation_df[citation_df.Parenthetical_Counts > 0]
citation_df_no_blanks

## 7. Analyze Keywords in Context
This section uses frequency plots and regression analysis to determine whether rhetorical analysis term usage and/or citation practice usage are good indicators of score. 

### Rhetorical Terms Regression Analysis

In [ ]:
#We need the metadata and text with newlines here; we'll also take the nostops text for further count analysis
rhetorical_keywords_df_full_texts = clean_essay_grades_df[['ID', 'Section', 'Portfolio_Score', 'Text_Newlines', 'NoStops_Text']].copy()

#Add ID and score in one column
rhetorical_keywords_df_full_texts['Score_ID'] = 'Score: ' + rhetorical_keywords_df_full_texts['Portfolio_Score'].astype(str) + ', ID:' + rhetorical_keywords_df_full_texts['ID'].astype(str)

#Check new df
rhetorical_keywords_df_full_texts.head()


In [ ]:
#Count usage of each term in each essay
pathos_counts = rhetorical_keywords_df_full_texts['NoStops_Text'].str.count('pathos')
ethos_counts = rhetorical_keywords_df_full_texts['NoStops_Text'].str.count('ethos')
logos_counts = rhetorical_keywords_df_full_texts['NoStops_Text'].str.count('logos')

#Append each count to the dataframe
rhetorical_keywords_df_full_texts['Pathos_Counts'] = pathos_counts
rhetorical_keywords_df_full_texts["Ethos_Counts"] = ethos_counts
rhetorical_keywords_df_full_texts["Logos_Counts"] = logos_counts

#Get summ of all term usages
rhetorical_terms = ['Pathos_Counts', 'Ethos_Counts', 'Logos_Counts']
rhetorical_keywords_df_full_texts['Sum_Terms'] = rhetorical_keywords_df_full_texts[rhetorical_terms].sum(axis=1)

rhetorical_keywords_df_full_texts

In [ ]:
#Chart number of times each term was used in each essay 
#Create bar graph
#https://plotly.com/python/bar-charts/
import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Bar(name='Pathos Counts', x=rhetorical_keywords_df_full_texts["Score_ID"], y=rhetorical_keywords_df_full_texts["Pathos_Counts"]),
    go.Bar(name='Ethos Counts', x=rhetorical_keywords_df_full_texts["Score_ID"], y=rhetorical_keywords_df_full_texts["Ethos_Counts"]),
    go.Bar(name='Logos Counts', x=rhetorical_keywords_df_full_texts["Score_ID"], y=rhetorical_keywords_df_full_texts["Logos_Counts"]),
    go.Bar(name='All Term Counts', x=rhetorical_keywords_df_full_texts["Score_ID"], y=rhetorical_keywords_df_full_texts["Sum_Terms"]),

])

# Change the bar mode
fig.update_layout(title_text='Counts of Each Rhetorical Term in Each Essay')
fig.update_layout(barmode='stack')
fig.show()

In [ ]:
#Check if amount of all term usage is indicative of grade
#Based on results, there is little relationship between amount of rhetorical terms used and grade...at least between A and B range essays)

#Create arrays of independent (x) and dependent (y) variables
x = np.array(rhetorical_keywords_df_full_texts['Portfolio_Score'])
y = np.array(rhetorical_keywords_df_full_texts['Sum_Terms'])

#Return key values of linear regression
slope, intercept, r, p, std_err = stats.linregress(x, y)

result = stats.linregress(x, y)

plt.plot(x, y, 'o', label='Student Essay Data', color = 'g')
plt.plot(x, result.intercept + result.slope*x, 'r', label='Predicted Score')
plt.xlabel("Paper Score")
plt.ylabel("Rhetorical Term Counts")
plt.legend()
plt.show()

print(f"R-squared for Rhetorical Terms: {result.rvalue**2:}")



In [ ]:
#Check if amount of usages of pathos is indicative of grade
#Create arrays of independent (x) and dependent (y) variables
x = np.array(rhetorical_keywords_df_full_texts['Portfolio_Score'])
y = np.array(rhetorical_keywords_df_full_texts['Pathos_Counts'])

#Return key values of linear regression
slope, intercept, r, p, std_err = stats.linregress(x, y)

result = stats.linregress(x, y)

print(f"R-squared for Pathos Terms: {result.rvalue**2:}")

plt.plot(x, y, 'o', label='Student Essay Data')
plt.plot(x, result.intercept + result.slope*x, 'r', label='Predicted Score')
plt.legend()
plt.show()


#Check if amount of usages of logos is indicative of grade
#Create arrays of independent (x) and dependent (y) variables
x = np.array(rhetorical_keywords_df_full_texts['Portfolio_Score'])
y = np.array(rhetorical_keywords_df_full_texts['Logos_Counts'])


#Return key values of linear regression
slope, intercept, r, p, std_err = stats.linregress(x, y)

result = stats.linregress(x, y)

print(f"R-squared for Logos Terms: {result.rvalue**2:}")

plt.plot(x, y, 'o', label='Student Essay Data')
plt.plot(x, result.intercept + result.slope*x, 'r', label='Predicted Score')
plt.legend()
plt.show()


#Check if amount of usages of ethos is indicative of grade
#Create arrays of independent (x) and dependent (y) variables
x = np.array(rhetorical_keywords_df_full_texts['Portfolio_Score'])
y = np.array(rhetorical_keywords_df_full_texts['Ethos_Counts'])


#Return key values of linear regression
slope, intercept, r, p, std_err = stats.linregress(x, y)

result = stats.linregress(x, y)

print(f"R-squared for Ethos Terms: {result.rvalue**2:}")

plt.plot(x, y, 'o', label='Student Essay Data')
plt.plot(x, result.intercept + result.slope*x, 'r', label='Predicted Score')
plt.legend()
plt.show()

In [ ]:
#Plot # paragraphs in which terms were used vs. essay grade
##In other words, do more successful writers use terms in multiple paragrpahs (indicating more coherence)?

#Count number of paragraphs where terms used and append to new dataframe
new_Series = rhetorical_keywords_paragraphs_df_no_blanks['Score_ID'].value_counts(ascending=True)
df3 = pd.DataFrame(new_Series).reset_index()
df3

df3.rename(columns={"index": "Score_ID", "Score_ID": "Paragraph_Counts"}, errors="raise", inplace=True)
df3[['ID','Score']] = df3.Score_ID.str.split(", ",expand=True)

df3

#Plot paragraph counts per paper
fig = go.Figure(data=[
    go.Bar(name='Paragraph Counts', x=df3["Score_ID"], y=df3["Paragraph_Counts"]),

])
# Change the bar mode
fig.update_layout(title_text='Number of Paragraphs Where Rhetorical Terms Were Used')
fig.update_layout(barmode='stack', xaxis={'categoryorder':'category ascending'})
fig.show()

In [ ]:
#Create new df for paragraph counts
df3[['Score','ID']] = df3.Score_ID.str.split(", ",expand=True)
df3['Score'] = df3['Score'].map(lambda x: x.lstrip('Score: '))
df3 = df3[['Score','Paragraph_Counts']].copy()
df3 = df3.apply(pd.to_numeric)
df3

In [ ]:
#Check if amount of paragraph term usage is indicative of grade
#Based on results (r = .08, there is little relationship between amount of rhetorical terms used and grade...at least between A and B range essays)
from scipy import stats

#Check if amount of usages of all terms per paragraph is indicative of grade
#Create arrays of independent (x) and dependent (y) variables


x = np.array(df3['Score'])
y = np.array(df3['Paragraph_Counts'])

#Return key values of linear regression
slope, intercept, r, p, std_err = stats.linregress(x, y)

result = stats.linregress(x, y)

print(f"R-squared for Terms Per Paragraph: {result.rvalue**2:}")

plt.plot(x, y, 's', label='original data')
plt.plot(x, result.intercept + result.slope*x, 'y', label='fitted line')
plt.legend()
plt.show()

### Citation Practice Regression Analysis


In [ ]:
#Using FULL TEXTS Get any text inside parentheticals and count of parentheticals and append to dataframe
#https://stackoverflow.com/questions/24696715/regex-for-match-parentheses-in-python
parentheticals = r'(?<=\().*?(?=\))'

parenthetical_matches = []
parenthetical_counts = []

citation_df_full_texts = clean_essay_grades_df[['ID', 'Section', 'Portfolio_Score','Text']].copy()
for text in citation_df_full_texts['Text']:
  matches = re.findall(parentheticals, text)
  parenthetical_matches.append(matches)
  parenthetical_counts.append(len(matches))

citation_df_full_texts["Parentheticals"] = parenthetical_matches
citation_df_full_texts['Parenthetical_Counts'] = parenthetical_counts
citation_df_full_texts

In [ ]:
#Add ID and score in one column
citation_df_full_texts['Score_ID'] = 'Score: ' + citation_df_full_texts['Portfolio_Score'].astype(str) + ', ID:' + citation_df_full_texts['ID'].astype(str)

In [ ]:
#Chart number of times parentheticals were used in each essay 
#Create bar graph
#https://plotly.com/python/bar-charts/
import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Bar(name='Parenthetical_Tags', x=citation_df_full_texts["Score_ID"], y=citation_df_full_texts["Parenthetical_Counts"])
])
# Change the bar mode
fig.update_layout(title_text='Counts of Parentheticals Used in Each Essay')
fig.update_layout(barmode='stack')
fig.show()

In [ ]:
#Regression: Parentheticals vs. Grade

#Check if amount of all term usage is indicative of grade
#Based on results (r = .08, there is little relationship between amount of rhetorical terms used and grade...at least between A and B range essays)
from scipy import stats


#Create arrays of independent (x) and dependent (y) variables
x = np.array(citation_df_full_texts['Portfolio_Score'])
y = np.array(citation_df_full_texts['Parenthetical_Counts'])

#Return key values of linear regression
slope, intercept, r, p, std_err = stats.linregress(x, y)

result = stats.linregress(x, y)


plt.plot(x, y, 's', label='Student Essay Data', color = 'y')
plt.plot(x, result.intercept + result.slope*x, 'r', label='Predicted Score')
plt.xlabel("Paper Score")
plt.ylabel("Parenthetical Citation Counts")
plt.legend()
plt.show()

print(f"R-squared for Parenthetical Citations: {result.rvalue**2:}")



In [ ]:
#Plot # paragraphs in which terms were used vs. essay grade
##In other words, do more successful writers use terms in multiple paragrpahs (indicating more coherence)?

#Count number of paragraphs where terms used and append to new dataframe
new_Series = citation_df['Score_ID'].value_counts(ascending=True)
df3 = pd.DataFrame(new_Series).reset_index()
df3

df3.rename(columns={"index": "Score_ID", "Score_ID": "Paragraph_Counts"}, errors="raise", inplace=True)
df3[['ID','Score']] = df3.Score_ID.str.split(", ",expand=True)

df3

#Plot paragraph counts per paper
fig = go.Figure(data=[
    go.Bar(name='Paragraph Counts', x=df3["Score_ID"], y=df3["Paragraph_Counts"]),

])
# Change the bar mode
fig.update_layout(title_text='Number of Paragraphs Where Citation Terms Were Used')
fig.update_layout(barmode='stack', xaxis={'categoryorder':'category ascending'})
fig.show()

In [ ]:
df3[['Score','ID']] = df3.Score_ID.str.split(", ",expand=True)
df3['Score'] = df3['Score'].map(lambda x: x.lstrip('Score: '))
df3 = df3[['Score','Paragraph_Counts']].copy()
df3 = df3.apply(pd.to_numeric)
df3

In [ ]:
#Check if amount of paragraph term usage is indicative of grade
#Based on results (r = .08, there is little relationship between amount of rhetorical terms used and grade...at least between A and B range essays)
from scipy import stats

#Check if amount of usages of all terms per paragraph is indicative of grade
#Create arrays of independent (x) and dependent (y) variables


x = np.array(df3['Score'])
y = np.array(df3['Paragraph_Counts'])

#Return key values of linear regressionslope, intercept, r, p, std_err = stats.linregress(x, y)

result = stats.linregress(x, y)


plt.plot(x, y, 'o', label='Student Essay Data')
plt.plot(x, result.intercept + result.slope*x, 'r', label='Predicted Score')
plt.xlabel("Paper Score")
plt.ylabel("Paragraph Counts of Parenthetical Citations")
plt.legend()
plt.show()

print(f"R-squared for Parenthetical Citations Per Paragraph: {result.rvalue**2:}")


The analysis below uses output from the DocuScope Corpus Analysis platform. This platform is freely available for download from Carnegie Melon University:https://www.cmu.edu/dietrich/english/research-and-publications/docuscope.html
 

DocuScope is a dictionary-based tool that "tags" words and phrases in texts based on its 50+ categories of rhetorical primers. The tool might tag the words “according to,” and “is proposing that” as evidence that the student is engaging in citation, for example. In aggregate, these counts can indicate to what degree each text in a corpus contains language indicating a particular rhetorical effect. Our interest in this case is the language DocuScope has tagged as indicating "Citation" is occuring; this language can be isolated from the CSV generated from the DocuScope tool, and an example can be found on this Github repository. 

In [ ]:
##Get current working directory 
path = os.getcwd()
print(path)

#Change working directory
path = os.chdir("/Users/megankane/Desktop")

In [ ]:
lats_df = pd.read_csv('DIMENSION_C_deidentified_texts_citation_clusters.csv')
lats_df


In [ ]:
#Make ID document to merge docuscope lats with
ids = clean_essay_grades_df[['ID', 'Portfolio_Score']].copy()
ids

#Rename filename column to id and merge target and LAT tables based on ID
lats_df.rename(columns={"Filename": "ID"}, inplace=True)
lats_df['ID'] = lats_df['ID'].map(lambda x: x.rstrip('.txt'))
lats_df['ID'] = lats_df['ID'].astype('float')
merged_lat_df = pd.merge(ids, lats_df, on='ID')

#Add ID and score in one column
merged_lat_df['Score_ID'] = 'Score: ' + merged_lat_df['Portfolio_Score'].astype(str) + ', ID:' + merged_lat_df['ID'].astype(str)
merged_lat_df

In [ ]:
#Chart number of times all terms were used in each essay 
#Create bar graph
#https://plotly.com/python/bar-charts/
import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Bar(name='Citations_Tags', x=merged_lat_df["Score_ID"], y=merged_lat_df["Citation"])
])
# Change the bar mode
fig.update_layout(title_text='Counts of All Citation Cluster Terms Used in Each Essay')
fig.update_layout(barmode='stack')
fig.show()

In [ ]:
#Regression: Citation vs. Grade

#Check if amount of all term usage is indicative of grade
#Based on results (r = .08, there is little relationship between amount of rhetorical terms used and grade...at least between A and B range essays)
from scipy import stats


#Create arrays of independent (x) and dependent (y) variables
x = np.array(merged_lat_df['Portfolio_Score'])
y = np.array(merged_lat_df['Citation'])

#Return key values of linear regression
slope, intercept, r, p, std_err = stats.linregress(x, y)

result = stats.linregress(x, y)


plt.plot(x, y, 'o', label='Student Essay Data', color = 'k')
plt.plot(x, result.intercept + result.slope*x, 'r', label='Predicted Score')
plt.xlabel("Paper Score")
plt.ylabel("DocuScope Citation Counts")
plt.legend()
plt.show()

print(f"R-squared for DocuScope Citations: {result.rvalue**2:}")


In [ ]:
#Chart number of times each citation dimension was used in each essay 
#Create bar graph
#https://plotly.com/python/bar-charts/
import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Bar(name='CitationAuthority', x=merged_lat_df["Score_ID"], y=merged_lat_df["CitationAuthority"]),
    go.Bar(name='CitationControversy', x=merged_lat_df["Score_ID"], y=merged_lat_df["CitationControversy"]),
    go.Bar(name='CitationGeneric', x=merged_lat_df["Score_ID"], y=merged_lat_df["CitationGeneric"]),
    go.Bar(name='CitationHedged', x=merged_lat_df["Score_ID"], y=merged_lat_df["CitationHedged"]),
    go.Bar(name='CitationNegative', x=merged_lat_df["Score_ID"], y=merged_lat_df["CitationNegative"]),
    go.Bar(name='CitationNeutral', x=merged_lat_df["Score_ID"], y=merged_lat_df["CitationNeutral"]),
    go.Bar(name='CitationSpeakerLookMood', x=merged_lat_df["Score_ID"], y=merged_lat_df["CitationSpeakerLookMood"]),
    go.Bar(name='UncertainCitation', x=merged_lat_df["Score_ID"], y=merged_lat_df["UncertainCitation"]),

])


# Change the bar mode
fig.update_layout(title_text='Counts of Each Rhetorical Term in Each Essay')
fig.update_layout(barmode='stack')
fig.show()